# Task 05 - Non-parametric probability density estimation - Parzen window
## Pattern Recognition and Machine Learning

Copy and import needed files/methods from previous assignment to this directory. 
Adding path to the previous assignment is not sufficient. Upload system
requires your code to be self contained.

In [ ]:
# uncomment following for interactive matplotlib
# %matplotlib notebook

from parzen import *
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import copy
from PIL import Image

from bayes import *

## init

In [ ]:
def unwrap(data):
    """
    Simple "hack" for preparing data from *.mat files
    """
    try:
        while (len(data) == 1) and (len(data.shape) > 0):
            data = data[0]
        for key in list(data.dtype.names):
            data[key] = unwrap(data[key])
    except:
        pass
    return data

def ndarray2dict(data, indexes=None):
    outputs = {}
    for key in list(data.dtype.names):
        value = unwrap(data[key])
        try:
            if len(value.shape) > 0:
                value = np.atleast_2d(value)
        except:
            pass
        outputs[key] = value
    if indexes is not None:
        for key in indexes:
            outputs[key] -= 1
    return outputs
    
data = scipy.io.loadmat("data_33rpz_parzen.mat")
tst = ndarray2dict(unwrap(data["tst"]), ['labels'])
trn = ndarray2dict(unwrap(data["trn"]), ['labels'])

## Tasks, part 1
#### measurements

In [ ]:
x = compute_measurement_lr_cont(trn['images'])

# splitting the trainning data into classes
idxs = np.squeeze(trn['labels'])

raise NotImplementedError("You have to implement the rest.")
xA = ...
xC = ...

#### computing the histograms of training data

In [ ]:
bins_A, centers_A = np.histogram(xA, 20)
bins_A = bins_A / (np.sum(bins_A)*(centers_A[1]-centers_A[0]))

bins_C, centers_C = np.histogram(xC, 20)
bins_C = bins_C / (np.sum(bins_C)*(centers_C[1]-centers_C[0]))

#### estimating conditional probability using Parzen window

In [ ]:
x_range = np.atleast_2d(np.arange(np.min(xA),np.max(xA),100))
h = np.array([[100, 500, 1000, 2000]])

raise NotImplementedError("You have to implement the rest.")
p = ...

#### visualisation

In [ ]:
# plots of the estimates
plt.figure(figsize=(15,10))

for idx in range(4):
    cur_h = h[0,idx]
    plt.subplot(2,2,idx+1)
    width = centers_A[1]-centers_A[0]
    plt.bar(centers_A[:-1] + width/2, bins_A, width=width*0.8)
    plt.plot(x_range.T, ..., 'r', linewidth=2)
    plt.title('h = {}'.format(cur_h))
    plt.xlabel('x')
    plt.ylabel('p(x|A)')
    plt.ylim([0, 4.5e-4])
    plt.grid('on')
plt.savefig('parzen_estimates.png')

## Tasks, part 2
#### 10-fold cross-validation init

In [ ]:
# h_range = np.arange(100,1000+1e-8,50)
h_range = np.linspace(100,1000,19)
num_folds = 10;

#### class A cross-validation

In [ ]:
np.random.seed(42)   # needed only for upload system, to test the correctness of the code

num_data = xA.size
itrn, itst = crossval(num_data, num_folds)

raise NotImplementedError("You have to implement the rest.")
Lh = ...

#### optimal value of parameter h

In [ ]:
raise NotImplementedError("You have to implement the rest.")
h_bestA = ...
Lh_bestA = ...

#### plots of optimal h

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)
plt.plot(h_range.T, Lh.T)
plt.plot(h_bestA, Lh_bestA, 'or')
bottom, _ = plt.ylim()
plt.plot([h_bestA, h_bestA], [bottom, Lh_bestA], '--r')
plt.title('10-fold cross-validation')
plt.xlabel('h')
plt.ylabel('L(h)')
plt.grid('on')

raise NotImplementedError("You have to implement the rest.")
p = ...

plt.subplot(1,2,2)
width = centers_A[1]-centers_A[0]
plt.bar(centers_A[:-1] + width/2, bins_A, width=width*0.9)
plt.plot(x_range.T, p.T, 'r', linewidth=2)
plt.grid('on')
plt.title('Best bandwidth h for class A')
plt.xlabel('x')
plt.ylabel('p(x|A)')
plt.savefig('optimal_h_classA.png')

#### class C cross-validation

In [ ]:
x_range = np.arange(np.min(xC),np.max(xC),100)

np.random.seed(42)   # needed only for upload system, to test the correctness of the code

num_data = xC.size
itrn, itst = crossval(num_data, num_folds)

Lh = np.zeros([1, h_range.size])
for h_iter in range(h_range.size):
    Lh[0,h_iter] = compute_Lh(itrn, itst, xC, h_range[h_iter])

#### optimal value of parameter h

In [ ]:
h_bestC = opt.fminbound(lambda h: -compute_Lh(itrn, itst, xC, h), h_range[0], h_range[-1])
Lh_bestC = compute_Lh(itrn, itst, xC, h_bestC)

#### plots of optimal h

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)
plt.plot(h_range.T, Lh.T)
plt.plot(h_bestC, Lh_bestC, 'or')
bottom, _ = plt.ylim()
plt.plot([h_bestC, h_bestC], [bottom, Lh_bestC], '--r')
plt.title('10-fold cross-validation')
plt.xlabel('h')
plt.ylabel('L(h)')
plt.grid('on')

raise NotImplementedError("You have to implement the rest.")
p = ...

plt.subplot(1,2,2)
width = centers_C[1]-centers_C[0]
plt.bar(centers_C[:-1] + width/2, bins_C, width=width*0.9)
plt.plot(x_range.T, p.T, 'r', linewidth=2)
plt.grid('on')
plt.title('Best bandwidth h for class C')
plt.xlabel('x')
plt.ylabel('p(x|C)')
plt.savefig('optimal_h_classC.png')

## Bayesian classifier

In [ ]:
x_test = compute_measurement_lr_cont(tst['images'])

# computing a priori probabilities

raise NotImplementedError("You have to implement the rest.")
pA = ...
pC = ...

labels = classify_bayes_parzen( ... )

#### visualisation

In [ ]:
show_classification(tst['images'], labels, 'AC')

# classification error
raise NotImplementedError("You have to implement the rest.")
bayes_error = ...
print(bayes_error)